In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

np.random.seed(4)

C:\Users\peng-weiCHEN\anaconda3\envs\psc\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
root_path=''

In [20]:
X_train = np.load(root_path+'X_train_ori.npy')
X_test = np.load(root_path+'X_test_ori.npy')
#eye_1s = np.load('data_1.npy')
y_train = np.load(root_path+'Y_train_ori.npy') #Si on veut que valence, mettre [:,0]. Si on veut arousal, mettre [:,1]. Si on veut les deux, rien mettre
y_test = np.load(root_path+'Y_test_ori.npy') #idem
#labels_1 = np.load('label_1.npy')

In [21]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
#176: nombres de vidéos ,500: nombre de times samples, 76: nombre de charactéristiques (38 pour chaque oeil)

(174, 16000, 94)
(174, 2)
(44, 16000, 94)
(44, 2)


In [22]:
#cellule optionnelle

class_nb = 7 # Convert labels from [0,1] to {0,1} or {0, 1, 2}

if class_nb == 3:
    y_train[y_train < 0.2] = 0
    y_train[(y_train > 0.7)] = 2
    y_train[(y_train >= 0.2) * (y_train <= 0.7)] = 1

    y_test[y_test <= 0.2] = 0
    y_test[(y_test > 0.7)] = 2
    y_test[(y_test > 0.2) * (y_test <= 0.7)] = 1

elif class_nb == 2:
    y_train[y_train <= 0.51] = 0
    y_train[y_train > 0.51] = 1
    
    y_test[y_test <= 0.51] = 0
    y_test[y_test > 0.51] = 1
else:
    pass

In [23]:
%%time

nb_pca = 7 #7 avant, je teste avec 5 même si ça explique moins de variance
###### Train #####
print("Initializing PCA with train set")
x = X_train.reshape(X_train.shape[0] * X_train.shape[1], X_train.shape[2])
# Normalisation
standard = StandardScaler()
x = standard.fit_transform(x)

# PCA
pca = PCA(n_components=nb_pca)
x = pca.fit_transform(x) #normalisation

X_train = x.reshape(X_train.shape[0], X_train.shape[1], nb_pca)

###### Test #####
print("Test processing...", end='')
x = X_test.reshape(X_test.shape[0] * X_test.shape[1], X_test.shape[2])
x = standard.transform(x)

# PCA
x = pca.transform(x)

X_test = x.reshape(X_test.shape[0], X_test.shape[1], nb_pca)
print("Finished!")

# Show PCA explanation ratio
print("Percent of values explained by axe:", pca.explained_variance_ratio_)
print("Percent of values explained(total):", pca.explained_variance_ratio_.sum())

Initializing PCA with train set
Test processing...Finished!
Percent of values explained by axe: [0.11925205 0.08457853 0.0744392  0.07118035 0.05953302 0.04815948
 0.04437237]
Percent of values explained(total): 0.5015149975543809
Wall time: 26.4 s


# A faire varier
*  nb_clusters 
* metric = "dtw" ou "euclidean"
* méthode pour définir la valeur dans un cluster
* valence ou arousal ou les deux pour la fonction choix (1ere ou deuxième colonne de y_train)

In [72]:
##### KMeans #####
nb_clusters = 4

print("Running KMeans Algorithm...", end='')
km = TimeSeriesKMeans(n_clusters=nb_clusters, metric="dtw", max_iter=5,n_jobs=-1, random_state=0).fit(X_train)
print("Finished!")

Running KMeans Algorithm...Finished!


In [73]:
class_train = km.predict(X_train)
dic = {} #dic[i]: ensemble des labels des vidéos du cluster i
for i in np.unique(class_train):
    dic[i] = []
    for j in range(len(class_train)):
        if class_train[j] == i:
            dic[i].append(y_train[j])
    dic[i] = np.array(dic[i])

In [74]:
#Pour vérifier si la répartition dans les clusters est homogène

for i in dic.keys():
    print("Nombre de ts dans la classe {} : {}".format(i,len(dic[i]))) 
    print("Valence moyenne dans la classe : ",np.mean(dic[i][:,0])) #enlever [:,0] si on n'a pris que la valence au début dans y_train
    print("Arousal moyenne dans la classe : ",np.mean(dic[i][:,1])) 

#problème: on ne réussit pas à faire des clusters "neutre", "positif", "négatif" 

Nombre de ts dans la classe 0 : 35
Valence moyenne dans la classe :  0.5380952380952381
Arousal moyenne dans la classe :  0.4809523809523809
Nombre de ts dans la classe 1 : 14
Valence moyenne dans la classe :  0.5833333333333333
Arousal moyenne dans la classe :  0.5357142857142857
Nombre de ts dans la classe 2 : 90
Valence moyenne dans la classe :  0.6518518518518518
Arousal moyenne dans la classe :  0.4981481481481482
Nombre de ts dans la classe 3 : 35
Valence moyenne dans la classe :  0.5952380952380953
Arousal moyenne dans la classe :  0.5476190476190477


# A définir
La fonction choix

In [75]:
#Fonction de choix pour une régression

def choix(lis): #lis: liste des labels en valence-arousal des vidéos du cluster i (les labels sont soit juste val, soit juste ar, soit tableau [val,ar])
    #ie si on prend valence&arousal, lis = [[val_1,arousal_1], ..., [val_p,arousal_p]]
    # Changez le code!!! Différents méthodes pour définir la valeur dans un cluster.
    #proposition: renvoyer la valence et l'arousal moyennes des gens du cluster sous la forme d'un tableau [valence_predite,arousal_predite]=[valence_moy_cluster,arousal_moy_cluster]
    return np.mean(lis,axis=0)


class_test = km.predict(X_test) #avant, np.squeeze(km.predict(X_test)), mais inutile?
prediction_test = []
for i in class_test:
    prediction_test.append(choix(dic[i])) # C'est là qu'on définit le choix
                                          # dic[i] est une listes des valeurs réelles dont leur données sont dans 
                                          # la classe i.
prediction = np.array(prediction_test)

In [76]:
erreurValence = (np.mean((prediction[:,0]*6 - y_test[:,0]*6)**2))
erreurArousal = (np.mean((prediction[:,1]*6 - y_test[:,1]*6)**2))
print("nb_clusters = {}".format(nb_clusters))
print("Erreur sur la valence : ",erreurValence)
print("Erreur sur l'arousal : ",erreurArousal)

nb_clusters = 4
Erreur sur la valence :  3.724308160974827
Erreur sur l'arousal :  3.2356182001420097
